# Poisoning of CICIDS-2017 dataset and detecting poisoning using LFA and defending with LSD/CSD/KSSD

## Setup and import required modules

In [ ]:
!pip3 install scikit-learn-intelex

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
from __future__ import print_function
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn import model_selection
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from scipy.sparse import csr_matrix, vstack, hstack
from scipy.sparse import coo_matrix
from keras.preprocessing.text import one_hot
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
#from sklearn.semi_supervised import label_propagation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
import math
#import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation , Flatten
from sklearn.metrics import log_loss
# from keras.optimizers import SGD
# from keras.layers.normalization import BatchNormalization
# from keras.layers.convolutional import UpSampling2D
# from keras.layers.convolutional import Conv2D, MaxPooling2D, MaxPooling1D
# from keras.layers.embeddings import Embedding
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPooling1D
from tensorflow.keras.layers import Embedding
from scipy import sparse
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay

#import random
import sklearn
from sklearn.metrics.pairwise import manhattan_distances
from keras.models import Model
from keras.layers import  Conv1D, multiply, GlobalMaxPool1D, Input , Lambda
import time
import argparse
#import math
from numpy import *
import os.path as osp
import scipy.sparse as sp
import pickle
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import precision_recall_curve
def plot_conf_matrix(Y_val, Y_val_preds):
    """
    Plots a nice looking confusion matrix using Seaborn's heatmap()
    """
    sns.set(font_scale=1.5)
    fig, ax = plt.subplots(figsize=(3,3))
    ax = sns.heatmap(confusion_matrix(Y_val, Y_val_preds),
                     annot=True,
                     cbar=False,
                     fmt='d')
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")

In [ ]:
def plot_conf_matrix_from_matrix(cm, title):
    """
    Plots a nice looking confusion matrix using Seaborn's heatmap()
    """
    sns.set(font_scale=1.5)
    fig, ax = plt.subplots(figsize=(3,3))
    ax = sns.heatmap(cm,
                     annot=True,
                     cbar=False,
                     fmt='d')
    plt.title(label=title)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")

Load Data

In [ ]:
# import pandas as pd
# input_data = pd.read_csv('/content/drive/MyDrive/CICIDS/Converted-Processed-CICIDS.csv', index_col='Unnamed: 0')

# input_tables = input_data.copy()
# input_tables.reset_index(inplace=True)
# input_tables.drop(['index'], axis=1, inplace=True)

In [ ]:
dl2 = input_tables.copy(deep=True)
print("Pre-conversion values:\n", dl2['Label'].value_counts())
for index, row in dl2.iterrows():
  if row['Label'] == 'BENIGN':
    dl2.at[index, 'Label'] = 0
  else:
    dl2.at[index, 'Label'] = 1
print("")
print("Converted values:\n", dl2['Label'].value_counts())
dl2 = dl2['Label']
dl2 = dl2.astype('int')

## Main Program Start

### Scale Data

In [ ]:
# print("*********Semi-Supervised Deep Learning Based Approach Against Label Flipping Attack in Intrusion Detection System*****************")
# print("                                                                                                                               ")
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cols = data.select_dtypes(include=['float64','int64']).columns
sc_data = scaler.fit_transform(data.select_dtypes(include=['float64','int64']))
data = pd.DataFrame(sc_data, columns = cols)

### Split data to train, test, and validation

In [ ]:
seed = 10
test_size = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(data, dl2, test_size=test_size, random_state=seed)
test_size = 0.25
X_train, X_val, Y_train, Y_val= train_test_split(X_train, Y_train, test_size=test_size, random_state=seed)

In [ ]:
# row_val,column_val = X_train.shape
# print("row_train,column_train:", X_train.shape)
# print("                                                                   ")
# row_val,column_val = X_val.shape
# print("row_val,column_val:", X_val.shape)
# print("                                                                   ")
# row_test,column_test = X_test.shape
# print("row_test,column_test:", X_test.shape)
# print("                                                                   ")
# print("********************************************************************")

In [ ]:
## /content/drive/MyDrive/Dissertation/KNN
### Saving
x_train_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train.sav'
pickle.dump(X_train, open(x_train_filename, 'wb'))
x_test_filename = '/content/drive/MyDrive/Dissertation/KNN/X_test.sav'
pickle.dump(X_test, open(x_test_filename, 'wb'))
x_val_filename = '/content/drive/MyDrive/Dissertation/KNN/X_val.sav'
pickle.dump(X_val, open(x_val_filename, 'wb'))
y_train_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train.sav'
pickle.dump(Y_train, open(y_train_filename, 'wb'))
y_test_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_test.sav'
pickle.dump(Y_test, open(y_test_filename, 'wb'))
y_val_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_val.sav'
pickle.dump(Y_val, open(y_val_filename, 'wb'))

In [ ]:
# ### /content/drive/MyDrive/Dissertation/KNN
# ## Loading
# x_train_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train.sav'
# X_train = pickle.load(open(x_train_filename, 'rb'))
# x_test_filename = '/content/drive/MyDrive/Dissertation/KNN/X_test.sav'
# X_test = pickle.load(open(x_test_filename, 'rb'))
# x_val_filename = '/content/drive/MyDrive/Dissertation/KNN/X_val.sav'
# X_val = pickle.load(open(x_val_filename, 'rb'))
# y_train_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train.sav'
# Y_train = pickle.load(open(y_train_filename, 'rb'))
# y_test_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_test.sav'
# Y_test = pickle.load(open(y_test_filename, 'rb'))
# y_val_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_val.sav'
# Y_val = pickle.load(open(y_val_filename, 'rb'))

### Model Definition

In [ ]:
X_train_NoAttack = X_train.copy()
# Y_train_NoAttack=Y_train[:]
Y_train_NoAttack = Y_train.copy()

X_val_NoAttack = X_val.copy()
# Y_val_NoAttack=Y_val[:]
Y_val_NoAttack = Y_val.copy()

row_train_NoAttack, column_train_NoAttack = X_train_NoAttack.shape
model_main = Sequential()
# model_main.add(Embedding(row_train_NoAttack, 8, input_length=column_train_NoAttack))
model_main.add(Conv1D(16,2, strides=2, padding='same', input_shape=(69, 1)))
model_main.add(MaxPooling1D(pool_size=(4), strides=(2)))
model_main.add(Conv1D(32,2, strides=2, padding='same', input_shape=(69, 1)))
model_main.add(MaxPooling1D(pool_size=(4), strides=(2)))
model_main.add(Conv1D(64,2, strides=2, padding='same', input_shape=(69, 1)))
model_main.add(Flatten())
model_main.add(Dense(1, activation='sigmoid'))
model_main.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_main.fit(X_train_NoAttack, Y_train_NoAttack, epochs=200, verbose=1)

In [ ]:
Y_CNN_NoAttack = model_main.predict(X_test, verbose=0)
Y_predict_NoAttack = [0]*len(Y_CNN_NoAttack)

for i in range(len(Y_CNN_NoAttack)):
    if Y_CNN_NoAttack[i] < 0.5:
            Y_CNN_NoAttack[i] = 0
    else:
            Y_CNN_NoAttack[i] = 1

for i in range(len(Y_CNN_NoAttack)):
    Y_predict_NoAttack[i] = int(Y_CNN_NoAttack[i])

In [ ]:
### Saving
model_main_filename = '/content/drive/MyDrive/Dissertation/KNN/model_main.sav'
pickle.dump(model_main, open(model_main_filename, 'wb'))
X_train_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train_NoAttack.sav'
pickle.dump(X_train_NoAttack, open(X_train_NoAttack_filename, 'wb'))
Y_train_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train_NoAttack.sav'
pickle.dump(Y_train_NoAttack, open(Y_train_NoAttack_filename, 'wb'))
X_val_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/X_val_NoAttack.sav'
pickle.dump(X_val_NoAttack, open(X_val_NoAttack_filename, 'wb'))
Y_val_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_val_NoAttack.sav'
pickle.dump(Y_val_NoAttack, open(Y_val_NoAttack_filename, 'wb'))
Y_CNN_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_CNN_NoAttack.sav'
pickle.dump(Y_CNN_NoAttack, open(Y_CNN_NoAttack_filename, 'wb'))
Y_predict_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_predict_NoAttack.sav'
pickle.dump(Y_predict_NoAttack, open(Y_predict_NoAttack_filename, 'wb'))

In [ ]:
# ### Loading
# model_main_filename = '/content/drive/MyDrive/Dissertation/KNN/model_main.sav'
# model_main = pickle.load(open(model_main_filename, 'rb'))
# X_train_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train_NoAttack.sav'
# X_train_NoAttack = pickle.load(open(X_train_NoAttack_filename, 'rb'))
# Y_train_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train_NoAttack.sav'
# Y_train_NoAttack = pickle.load(open(Y_train_NoAttack_filename, 'rb'))
# X_val_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/X_val_NoAttack.sav'
# X_val_NoAttack = pickle.load(open(X_val_NoAttack_filename, 'rb'))
# Y_val_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_val_NoAttack.sav'
# Y_val_NoAttack = pickle.load(open(Y_val_NoAttack_filename, 'rb'))
# Y_CNN_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_CNN_NoAttack.sav'
# Y_CNN_NoAttack = pickle.load(open(Y_CNN_NoAttack_filename, 'rb'))
# Y_predict_NoAttack_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_predict_NoAttack.sav'
# Y_predict_NoAttack = pickle.load(open(Y_predict_NoAttack_filename, 'rb'))

### Result of Model without attack on X_test

In [ ]:
from sklearn.metrics import confusion_matrix
con_matrix = confusion_matrix(testing, Y_predict_NoAttack)
print(con_matrix)
TN_NoAttack, FP_NoAttack, FN_NoAttack, TP_NoAttack = confusion_matrix((Y_test.astype(int).tolist()), (Y_predict_NoAttack)).ravel()
print("TN_NoAttack=",TN_NoAttack)
print("FP_NoAttack=",FP_NoAttack)
print("FN_NoAttack=",FN_NoAttack)
print("TP_NoAttack=",TP_NoAttack)

In [ ]:
print("********************************Result of Model without attack******************************************************************")
# loss, accuracy = model_main.evaluate(X_train_NoAttack, Y_train_NoAttack, verbose=2)
#loss, accuracy = model_main.evaluate(X_train_NoAttack, Y_train_NoAttack, verbose=2)
loss, accuracy = model_main.evaluate(X_train_NoAttack, Y_train_NoAttack, verbose=2)

print('Accuracy for Train set: %f' % (accuracy*100))
print('Loss for Train set: %f' % (loss))
print("                                                                   ")

#loss, accuracy = model_main.evaluate(X_val_NoAttack, Y_val_NoAttack, verbose=2)
loss, accuracy = model_main.evaluate(X_val_NoAttack, Y_val_NoAttack, verbose=2)
print('Accuracy for Validation set: %f' % (accuracy*100))
print('Loss for Train Validation set: %f' % (loss))
print("                                                                   ")

loss, accuracy = model_main.evaluate(X_test, Y_test, verbose=2)
print('Accuracy for Test set: %f' % (accuracy*100))
print('Loss for Test set: %f' % (loss))
print("                                                                   ")

con_matrix = confusion_matrix(Y_test, Y_predict_NoAttack)
print(con_matrix, "\n")

TN_NoAttack, FP_NoAttack, FN_NoAttack, TP_NoAttack = confusion_matrix((Y_test.astype(int).tolist()), Y_predict_NoAttack).ravel()
print("TN_NoAttack:",TN_NoAttack)
print("FP_NoAttack:",FP_NoAttack)
print("FN_NoAttack:",FN_NoAttack)
print("TP_NoAttack:",TP_NoAttack)
print("                                                                   ")

if (FP_NoAttack + TN_NoAttack) > 0:
        FPR_NoAttack = FP_NoAttack/(FP_NoAttack + TN_NoAttack)
        print("The FPR_NoAttack result:", FPR_NoAttack)
            
if (FP_NoAttack + TN_NoAttack) > 0:
        TPR_NoAttack=TP_NoAttack/(TP_NoAttack+FN_NoAttack)
        print("The TPR_NoAttack result=", TPR_NoAttack)
            
if (TN_NoAttack + FP_NoAttack) > 0:
    TNR_NoAttack = TN_NoAttack/(TN_NoAttack + FP_NoAttack)
    print("The TNR_NoAttack result:", TNR_NoAttack)
            
if (FN_NoAttack + TP_NoAttack) > 0:
    FNR_NoAttack = FN_NoAttack/(FN_NoAttack + TP_NoAttack)
    print("The FNR_NoAttack result:", FNR_NoAttack)
            
if ((TN_NoAttack/(TN_NoAttack + FP_NoAttack)) + (TP_NoAttack/(TP_NoAttack + FP_NoAttack))) > 0:
    AUC_NoAttack = 1/(2*((TN_NoAttack/(TN_NoAttack + FP_NoAttack)) + (TP_NoAttack/(TP_NoAttack + FP_NoAttack))))
    print("The AUC_NoAttack result:", AUC_NoAttack)
            
if  (TP_NoAttack + TN_NoAttack + FP_NoAttack + FN_NoAttack) > 0:
    ACC_NoAttack = (TP_NoAttack + TN_NoAttack)/(TP_NoAttack + TN_NoAttack + FP_NoAttack + FN_NoAttack)
    print("The ACC_NoAttack result:", ACC_NoAttack)
            
if  ((TP_NoAttack + FP_NoAttack) * (TP_NoAttack + FN_NoAttack) * (TN_NoAttack + FP_NoAttack) * (TN_NoAttack + FN_NoAttack)) > 0:
    MCC_NoAttack = (TP_NoAttack * TN_NoAttack - FP_NoAttack * FN_NoAttack)/math.sqrt((TP_NoAttack + FP_NoAttack) * (TP_NoAttack + FN_NoAttack) * (TN_NoAttack + FP_NoAttack) * (TN_NoAttack + FN_NoAttack))
    print("The Matthews correlation coefficient result:", MCC_NoAttack)
print("                                                                                                                               ")
print("*****************************************************End of Without Attack part************************************************")

********************************Result of Model without attack******************************************************************
53077/53077 - 92s - loss: 0.0226 - acc: 0.9931 - 92s/epoch - 2ms/step
Accuracy for Train set: 99.313551
Loss for Train set: 0.022632
                                                                   
17693/17693 - 31s - loss: 0.0231 - acc: 0.9928 - 31s/epoch - 2ms/step
Accuracy for Validation set: 99.283755
Loss for Train Validation set: 0.023121
                                                                   
17693/17693 - 31s - loss: 0.0260 - acc: 0.9929 - 31s/epoch - 2ms/step
Accuracy for Test set: 99.287289
Loss for Test set:: 0.025963
                                                                   
[[452810   1654]
 [  2381 109304]] 

TN_NoAttack: 452810
FP_NoAttack: 1654
FN_NoAttack: 2381
TP_NoAttack: 109304
                                                                   
The FPR_NoAttack result: 0.003639452189832418
The TPR_NoAttack result= 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: RuntimeWarning: overflow encountered in long_scalars


## Label Flipping Attack

### Splitting data into malicious and benign

In [ ]:
print("*****************************************************Label Flipping Attack*****************************************************")
print("                                                                                                                               ")
#************************** 
# Finding Malware of Train data
# malware_train= sparse.lil_matrix(X_train)
malware_train = X_train.copy(deep=True)
y_malware_train = Y_train.copy(deep=True)
print("malware_train:", malware_train.shape)
cl_malware = list()
z_m = 0
count_m = 0
drop_list = []
for i, j in enumerate(Y_train):
#     if j == 1:
    if j == 0:
        drop_list.append(X_train.index[i])
        count_m = count_m + 1
    else:
        cl_malware.insert(z_m, 1)
        z_m = z_m + 1 
drop_list.sort(reverse=True)
malware_train.drop(drop_list, inplace=True)
y_malware_train.drop(drop_list, inplace=True)

#***************************
#Finding Benign of Train data
cl_X_train = list(Y_train) 
# benign_train=sparse.lil_matrix(X_train)
benign_train = X_train.copy(deep=True)
y_benign_train = Y_train.copy(deep=True)
print("benign_train:", benign_train.shape)
z_b = 0
count_b = 0
cl_benign = list()
drop_list = []
for i, j in enumerate(cl_X_train):
#     if j == 0:
    if j == 1:
        drop_list.append(X_train.index[i])
        count_b = count_b+1
    else:
        cl_benign.insert(z_b, 1)
        z_b = z_b+1
drop_list.sort(reverse=True)
benign_train.drop(drop_list, inplace=True)
y_benign_train.drop(drop_list, inplace=True)

print("***********Size of Each Data Part:**********")        
print("malware_train:", malware_train.shape)
print("benign_train:", benign_train.shape)
#***************************************************
row_malware_train, column_malware_train = malware_train.shape
# Number_of_flipped_label=int(row_malware_train)

X_train_LFA = X_train.copy(deep=True)
# Y_train_LFA=Y_train[:]
Y_train_LFA = Y_train.copy(deep=True)

row_train_LFA, column_train_LFA = X_train_LFA.shape

In [ ]:
### Saving
malware_train_filename = '/content/drive/MyDrive/Dissertation/KNN/malware_train.sav'
pickle.dump(malware_train, open(malware_train_filename, 'wb'))
y_malware_train_filename = '/content/drive/MyDrive/Dissertation/KNN/y_malware_train.sav'
pickle.dump(y_malware_train, open(y_malware_train_filename, 'wb'))
benign_train_filename = '/content/drive/MyDrive/Dissertation/KNN/benign_train.sav'
pickle.dump(benign_train, open(benign_train_filename, 'wb'))
y_benign_train_filename = '/content/drive/MyDrive/Dissertation/KNN/y_benign_train.sav'
pickle.dump(y_benign_train, open(y_benign_train_filename, 'wb'))
X_train_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train_LFA.sav'
pickle.dump(X_train_LFA, open(X_train_LFA_filename, 'wb'))
Y_train_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train_LFA.sav'
pickle.dump(Y_train_LFA, open(Y_train_LFA_filename, 'wb'))

In [ ]:
# ### loading
# malware_train_filename = '/content/drive/MyDrive/Dissertation/KNN/malware_train.sav'
# malware_train = pickle.load(open(malware_train_filename, 'rb'))
# y_malware_train_filename = '/content/drive/MyDrive/Dissertation/KNN/y_malware_train.sav'
# y_malware_train = pickle.load(open(y_malware_train_filename, 'rb'))
# benign_train_filename = '/content/drive/MyDrive/Dissertation/KNN/benign_train.sav'
# benign_train = pickle.load(open(benign_train_filename, 'rb'))
# y_benign_train_filename = '/content/drive/MyDrive/Dissertation/KNN/y_benign_train.sav'
# y_benign_train = pickle.load(open(y_benign_train_filename, 'rb'))
# X_train_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/X_train_LFA.sav'
# X_train_LFA = pickle.load(open(X_train_LFA_filename, 'rb'))
# Y_train_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/Y_train_LFA.sav'
# Y_train_LFA = pickle.load(open(Y_train_LFA_filename, 'rb'))

# row_train_LFA, column_train_LFA = X_train_LFA.shape

### Silhouette Sampling

In [ ]:
clusterer = KMeans(n_clusters=2, random_state=10)
#X=X_train_LFA.toarray()
X = X_train_LFA.copy(deep=True)
t0 = time.time()
cluster_labels = clusterer.fit_predict(X)
sample_silhouette_values = silhouette_samples(X, cluster_labels)
#print("sample_silhouette_values = ",sample_silhouette_values)
t1 = time.time()
print("Time for creating silhouette samples: ",t1-t0)

In [ ]:
### Saving silhouette_samples
silhouette_samples_filename = '/content/drive/MyDrive/Dissertation/KNN/silhouette_samples.pkl'
pickle.dump(sample_silhouette_values, open(silhouette_samples_filename, 'wb'))
clusterer_filename = '/content/drive/MyDrive/Dissertation/KNN/clusterer.pkl'
pickle.dump(clusterer, open(clusterer_filename, 'wb'))
cluster_labels_filename = '/content/drive/MyDrive/Dissertation/KNN/cluster_labels.pkl'
pickle.dump(cluster_labels, open(cluster_labels_filename, 'wb'))

In [ ]:
# ## Loading silhouette_samples
# silhouette_samples_filename = '/content/drive/MyDrive/Dissertation/KNN/silhouette_samples.pkl'
# sample_silhouette_values = pickle.load(open(silhouette_samples_filename, 'rb'))
# clusterer_filename = '/content/drive/MyDrive/Dissertation/KNN/clusterer.pkl'
# clusterer = pickle.load(open(clusterer_filename, 'rb'))
# cluster_labels_filename = '/content/drive/MyDrive/Dissertation/KNN/cluster_labels.pkl'
# cluster_labels = pickle.load(open(cluster_labels_filename, 'rb'))

In [ ]:
t0 = time.time()
print("Values:\n", Y_train_LFA.value_counts())
flipped_Y_train = list(Y_train_LFA)
counter = 0

### For CICIDS 2017 - replace silhouette_level with these values for different levels of poisoning
### 0.05 = Original label flipping level
### 0.15 = 5% label flipping
### 0.29 = 10% label flipping
### 0.335 = 15% label flipping
### 0.36 = 20% label flipping
### 0.405 = 25% label flipping

silhouette_level = 0.15
for new_index in range(row_train_LFA): 
    if (sample_silhouette_values[new_index]<silhouette_level):
        flipped_Y_train[new_index] = abs(flipped_Y_train[new_index]-1)     # flipped_Y_train[new_index]=1
        counter = counter + 1
# print(flipped_Y_train.count(0))
# print(flipped_Y_train.count(1))
print("Flipped counter:", counter)

t1 = time.time()
print("Time for Label Flipping Attack: ",t1-t0)
print("                                                         ") 

Values:
 0    1363891
1     334554
dtype: int64
Flipped counter: 76488
Time for Label Flipping Attack:  0.6521282196044922
                                                         


In [ ]:
### Saving
flipped_Y_train_filename = '/content/drive/MyDrive/Dissertation/KNN/5/flipped_Y_train.pkl'
pickle.dump(flipped_Y_train, open(flipped_Y_train_filename, 'wb'))

In [ ]:
# ## Loading
# flipped_Y_train_filename = '/content/drive/MyDrive/Dissertation/KNN/5/flipped_Y_train.pkl'
# flipped_Y_train = pickle.load(open(flipped_Y_train_filename, 'rb'))

### Training model with poisoned data

In [ ]:
#**************************************************************************
model_main_LFA_Final = Sequential()
#model_main_LFA_Final.add(Embedding(row_train_LFA, 8, input_length=column_train_LFA))
model_main_LFA_Final.add(Conv1D(16,2, strides=2, padding='same', input_shape=(69,1)))
model_main_LFA_Final.add(MaxPooling1D(pool_size=(4), strides=(2)))
model_main_LFA_Final.add(Conv1D(32,2, strides=2, padding='same', input_shape=(69,1)))
model_main_LFA_Final.add(MaxPooling1D(pool_size=(4), strides=(2)))
model_main_LFA_Final.add(Conv1D(64,2, strides=2, padding='same', input_shape=(69,1)))
model_main_LFA_Final.add(Flatten())
model_main_LFA_Final.add(Dense(1, activation='sigmoid'))
model_main_LFA_Final.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
#model_main_LFA_Final.fit(tf.convert_to_tensor(pd.DataFrame.sparse.from_spmatrix(X_train_LFA)), tf.convert_to_tensor(pd.DataFrame(flipped_Y_train)), epochs=200, verbose=0)
model_main_LFA_Final.fit(X_train_LFA, np.array(flipped_Y_train), epochs=200, verbose=1)

Epoch 1/200
53077/53077 [==============================] - 202s 4ms/step - loss: 0.1536 - acc: 0.9323
Epoch 2/200
53077/53077 [==============================] - 200s 4ms/step - loss: 0.1189 - acc: 0.9478
Epoch 3/200
53077/53077 [==============================] - 196s 4ms/step - loss: 0.1122 - acc: 0.9519
Epoch 4/200
53077/53077 [==============================] - 195s 4ms/step - loss: 0.1081 - acc: 0.9546
Epoch 5/200
53077/53077 [==============================] - 197s 4ms/step - loss: 0.1049 - acc: 0.9563
Epoch 6/200
53077/53077 [==============================] - 200s 4ms/step - loss: 0.1024 - acc: 0.9575
Epoch 7/200
53077/53077 [==============================] - 200s 4ms/step - loss: 0.1002 - acc: 0.9583
Epoch 8/200
53077/53077 [==============================] - 203s 4ms/step - loss: 0.0979 - acc: 0.9593
Epoch 9/200
53077/53077 [==============================] - 204s 4ms/step - loss: 0.0954 - acc: 0.9599
Epoch 10/200
53077/53077 [==============================] - 197s 4ms/step - loss: 

In [ ]:
Y_predict_LFA=model_main_LFA_Final.predict(X_test, verbose=0)
Y_predict_LFA_Final=[0]*len(Y_predict_LFA)

for i in range(len(Y_predict_LFA)):
    if Y_predict_LFA[i]<0.5:
            Y_predict_LFA[i] = 0
    else:
            Y_predict_LFA[i] = 1

for i in range(len(Y_predict_LFA)):
    Y_predict_LFA_Final[i] = int(Y_predict_LFA[i])  

In [ ]:
### Saving
model_main_LFA_Final_filename = '/content/drive/MyDrive/Dissertation/KNN/5/model_main_LFA_Final.pkl'
pickle.dump(model_main_LFA_Final, open(model_main_LFA_Final_filename, 'wb'))
Y_predict_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/5/Y_predict_LFA.pkl'
pickle.dump(Y_predict_LFA, open(Y_predict_LFA_filename, 'wb'))
Y_predict_LFA_Final_filename = '/content/drive/MyDrive/Dissertation/KNN/5/Y_predict_LFA_Final.pkl'
pickle.dump(Y_predict_LFA_Final, open(Y_predict_LFA_Final_filename, 'wb'))

INFO:tensorflow:Assets written to: ram://b080e728-6028-41a0-8b54-3fe25563fad6/assets


In [ ]:
# ### Loading
# model_main_LFA_Final_filename = '/content/drive/MyDrive/Dissertation/KNN/5/model_main_LFA_Final.pkl'
# model_main_LFA_Final = pickle.load(open(model_main_LFA_Final_filename, 'rb'))
# Y_predict_LFA_filename = '/content/drive/MyDrive/Dissertation/KNN/5/Y_predict_LFA.pkl'
# Y_predict_LFA = pickle.load(open(Y_predict_LFA_filename, 'rb'))
# Y_predict_LFA_Final_filename = '/content/drive/MyDrive/Dissertation/KNN/5/Y_predict_LFA_Final.pkl'
# Y_predict_LFA_Final = pickle.load(open(Y_predict_LFA_Final_filename, 'rb'))

### Result of Model with LFA

In [ ]:
print("********************************Result of Model with LFA attack **************************************************************")
print("                                                                                                                              ")
loss, accuracy = model_main_LFA_Final.evaluate(X_train_LFA, np.array(flipped_Y_train), verbose=2)
print('Accuracy for Train set: %f' % (accuracy*100))
print('Loss for Train set: %f' % (loss))
print("                                                                   ")

loss, accuracy = model_main_LFA_Final.evaluate(X_test, Y_test, verbose=2)
print('Accuracy for Test set: %f' % (accuracy*100))
print('Loss for Test set: %f' % (loss))
print("                                                                   ")

TN_LFA, FP_LFA, FN_LFA, TP_LFA = confusion_matrix(Y_test,  Y_predict_LFA_Final).ravel()
print("TN_LFA:", TN_LFA)
print("FP_LFA:", FP_LFA)
print("FN_LFA:", FN_LFA)
print("TP_LFA:", TP_LFA)
print("                                                                   ")

if (FP_LFA+TN_LFA) > 0:
        FPR_LFA=FP_LFA/(FP_LFA+TN_LFA)
        print("The FPR_LFA result=", FPR_LFA)
            
if (FP_LFA+TN_LFA) > 0:
        TPR_LFA = TP_LFA/(TP_LFA+FN_LFA)
        print("The TPR_LFA result:", TPR_LFA)
            
if (TN_LFA+FP_LFA) > 0:
    TNR_LFA = TN_LFA/(TN_LFA+FP_LFA)
    print("The TNR_LFA result:", TNR_LFA)
            
if (FN_LFA+TP_LFA) > 0:
    FNR_LFA = FN_LFA/(FN_LFA+TP_LFA)
    print("The FNR_LFA result:", FNR_LFA)
            
if ((TN_LFA/(TN_LFA+FP_LFA))+(TP_LFA/(TP_LFA+FP_LFA))) > 0:
    AUC_LFA = 1/(2*((TN_LFA/(TN_LFA+FP_LFA))+(TP_LFA/(TP_LFA+FP_LFA))))
    print("The AUC_LFA result:", AUC_LFA)
            
if  (TP_LFA+TN_LFA+FP_LFA+FN_LFA) > 0:
    ACC_LFA = (TP_LFA+TN_LFA)/(TP_LFA+TN_LFA+FP_LFA+FN_LFA)
    print("The ACC_LFA result:", ACC_LFA)
            
if  ((TP_LFA+FP_LFA)*(TP_LFA+FN_LFA)*(TN_LFA+FP_LFA)*(TN_LFA+FN_LFA)) > 0:
    MCC_LFA = (TP_LFA*TN_LFA-FP_LFA*FN_LFA)/math.sqrt((TP_LFA+FP_LFA)*(TP_LFA+FN_LFA)*(TN_LFA+FP_LFA)*(TN_LFA+FN_LFA))
    print("The Matthews correlation coefficient result:", MCC_LFA)
print("                                                                                                                               ")
print("************************************************End of Label Flipping Attack part**********************************************")

********************************Result of Model with LFA attack **************************************************************
                                                                                                                              
53077/53077 - 91s - loss: 0.0668 - acc: 0.9782 - 91s/epoch - 2ms/step
Accuracy for Train set: 97.822011
Loss for Train set: 0.066837
                                                                   
17693/17693 - 36s - loss: 0.4142 - acc: 0.9402 - 36s/epoch - 2ms/step
Accuracy for Test set: 94.024718
Loss for Test set: 0.414232
                                                                   
TN_LFA: 428869
FP_LFA: 25595
FN_LFA: 8234
TP_LFA: 103451
                                                                   
The FPR_LFA result= 0.05631909238135474
The TPR_LFA result: 0.9262747907060035
The TNR_LFA result: 0.9436809076186453
The FNR_LFA result: 0.07372520929399651
The AUC_LFA result: 0.28647700522917036
The ACC_LFA result: 0.9

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: overflow encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: RuntimeWarning: overflow encountered in long_scalars
